In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
exp = '3types_'
mod = 'rna'

In [4]:
import anndata as ad
import numpy as np
from scphere.model.vae import SCPHERE
from scphere.util.trainer import Trainer

/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/nlaa

In [5]:
neurips_data = '/home/users/nlaabid/neurips-data/full_data/cite.h5ad'
full_cite = ad.read_h5ad(neurips_data)
full_cite.var_names_make_unique()
full_cite

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 90261 × 14087
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [6]:
rna_data = full_cite[:,full_cite.var['feature_types']=='GEX']
rna_data

View of AnnData object with n_obs × n_vars = 90261 × 13953
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [7]:
prot_data = full_cite[:,full_cite.var['feature_types']=='ADT']
prot_data

View of AnnData object with n_obs × n_vars = 90261 × 134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [8]:
b = ['s1d1']
c = ['CD14+ Mono', 'Erythroblast', 'Proerythroblast']
data = rna_data[(rna_data.obs['batch'].isin(b))&(rna_data.obs['cell_type'].isin(c))]

data

View of AnnData object with n_obs × n_vars = 1062 × 13953
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [9]:
cell_types = data.obs['cell_type'].to_numpy()
np.savetxt('../demo-out/' + exp +
           'cite_cell_types.tsv',
           cell_types, fmt='%s')


In [10]:
x = data.X.toarray().astype(dtype='int')

In [11]:
batch = np.zeros(x.shape[0]) * -1

In [12]:
model = SCPHERE(n_gene=x.shape[1], n_batch=0, batch_invariant=False,
                z_dim=2, latent_dist='vmf',
                observation_dist='nb', seed=0)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-03-01 10:31:40,512: WARNING: 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-03-01 10:31:40,526: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-03-01 10:31:40,803: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


2022-03-01 10:31:41,401: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/scPhere-0.1.0-py3.6.egg/scphere/ops/ive.py:23: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [13]:
trainer = Trainer(model=model, x=x, batch_id=batch, max_epoch=250, mb_size=128, learning_rate=0.001)
trainer.train()

0 / 2000 {'Log-likelihood': -11770.075, 'ELBO': -11770.436, 'KL': 0.36058342}
50 / 2000 {'Log-likelihood': -5088.749, 'ELBO': -5091.254, 'KL': 2.5049827}
100 / 2000 {'Log-likelihood': -4490.131, 'ELBO': -4493.0312, 'KL': 2.9006224}
150 / 2000 {'Log-likelihood': -4467.678, 'ELBO': -4470.878, 'KL': 3.199923}
200 / 2000 {'Log-likelihood': -4582.501, 'ELBO': -4585.9844, 'KL': 3.483603}
250 / 2000 {'Log-likelihood': -4347.4907, 'ELBO': -4351.172, 'KL': 3.6810975}
300 / 2000 {'Log-likelihood': -4209.445, 'ELBO': -4213.2603, 'KL': 3.8153358}
350 / 2000 {'Log-likelihood': -4161.3613, 'ELBO': -4165.3193, 'KL': 3.9577942}
400 / 2000 {'Log-likelihood': -3979.674, 'ELBO': -3983.7898, 'KL': 4.115712}
450 / 2000 {'Log-likelihood': -4066.6755, 'ELBO': -4070.8848, 'KL': 4.209214}
500 / 2000 {'Log-likelihood': -4148.956, 'ELBO': -4153.2236, 'KL': 4.267685}
550 / 2000 {'Log-likelihood': -4050.8345, 'ELBO': -4055.2014, 'KL': 4.366825}
600 / 2000 {'Log-likelihood': -4027.4592, 'ELBO': -4031.891, 'KL': 4.4

In [14]:
save_path = '../demo-out/'

model_name = save_path + exp + mod + 'cite_250epoch'
model.save_sess(model_name)

In [15]:
z_mean = model.encode(x, batch)
np.savetxt(save_path + exp + mod +
           'cite_latent_250epoch.tsv',
           z_mean)


In [16]:
ll = model.get_log_likelihood(x, batch)
np.savetxt(save_path + exp + mod +
           'cite_ll_250epoch.tsv',
           z_mean)

In [17]:
# why is one float and one int?
# read study for data collection of scphere data